In [32]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import math

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

from nas_wot import score
from MNISTConvNet import MNISTConvNet

torch.set_printoptions(linewidth=120)
torch.set_grad_enabled(True)

In [33]:
import skopt
from skopt import gp_minimize, forest_minimize
from skopt.space import Real, Categorical, Integer
from skopt.plots import plot_convergence
from skopt.plots import plot_objective, plot_evaluations
from skopt.plots import plot_histogram, plot_objective_2D
from skopt.utils import use_named_args

In [34]:
dim_learning_rate = Real(low=1e-6, high=1e-2, prior='log-uniform',
                        name='learning_rate')

dim_num_conv_layers = Integer(low=1, high=3, name='num_conv_layers')

dim_num_fc_units = Integer(low=5, high=512, name='num_fc_units')

dim_dropout_rate = Real(low=1e-5, high=1e-2, prior='log-uniform',
                        name='dropout_rate')

dimensions = [dim_learning_rate,
              dim_num_conv_layers,
              dim_num_fc_units,
              dim_dropout_rate]

default_parameters = [1e-5, 1, 16, 1e-4]

In [85]:
train_set = torchvision.datasets.MNIST(
                        root='./data/MNIST',
                        train=True,           #Training Set of 60,000 images
                        download=True,
                        transform=transforms.Compose([
                            transforms.ToTensor(),
                            transforms.Normalize(0, 1)
                        ])
)

test_set = torchvision.datasets.MNIST(
                        root='./data/MNIST',
                        train=False,          #Test Set of 10,000 images
                        download=True,
                        transform=transforms.Compose([
                            transforms.ToTensor(),
                            transforms.Normalize(0, 1)
                        ])
)

train_loader = torch.utils.data.DataLoader(
    train_set, batch_size=100
)

test_loader = torch.utils.data.DataLoader(
    test_set, batch_size=100
)

In [36]:
def get_num_correct(preds, labels):
    return preds.argmax(dim=1).eq(labels).sum().item()

In [37]:
def train(model, lr, num_epoch, train_loader, test_loader):
    
    optimizer = optim.Adam(model.parameters(), lr=lr)

    for epoch in range(num_epoch):
        total_loss = 0
        total_correct = 0

        for batch in train_loader:
            images, labels = batch
            
            preds = model(images)
            loss = F.cross_entropy(preds, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            total_correct += get_num_correct(preds, labels)

        print('epoch:', epoch, 'total_correct:', total_correct, 'loss:', total_loss)

    print('Train Accuracy:', total_correct/len(train_set))

    test_loss = 0
    test_correct = 0

    for batch in test_loader:
        images, labels = batch

        preds = model(images)
        loss = F.cross_entropy(preds, labels)
        
        test_loss += loss.item()
        test_correct += get_num_correct(preds, labels)

    print('Test Accuracy:', test_correct/len(test_set))
    
    return test_correct/len(test_set)

In [38]:
best_accuracy = 0.0
best_model_path = './best_model.pth'

@use_named_args(dimensions=dimensions)
def fitness(learning_rate, num_conv_layers,
            num_fc_units, dropout_rate):

    print('\n\nlearning rate: {0:.1e}'.format(learning_rate))
    print('num_conv_layers:', num_conv_layers)
    print('num_fc_units:', num_fc_units)
    print('dropout_rate:', dropout_rate)
    
    model = MNISTConvNet(num_conv_layers=num_conv_layers,
                         num_fc_units=num_fc_units,
                         dropout_rate=dropout_rate)

    accuracy = train(model, learning_rate, 1, train_loader, test_loader)

    print('Accuracy:', accuracy)

    global best_accuracy

    if accuracy > best_accuracy:
        torch.save(model, best_model_path)
        best_accuracy = accuracy

    del model

    return -accuracy

In [39]:
fitness(x=default_parameters)



learning rate: 1.0e-05
num_conv_layers: 1
num_fc_units: 16
dropout_rate: 0.0001
epoch: 0 total_correct: 8990 loss: 1375.8962564468384
Train Accuracy: 0.14983333333333335
Test Accuracy: 0.1963
Accuracy: 0.1963


-0.1963

In [40]:
best_accuracy

0.1963

In [41]:
search_result = gp_minimize(func=fitness,
                            dimensions=dimensions,
                            acq_func='EI',
                            n_calls=12,
                            x0=default_parameters)



learning rate: 1.0e-05
num_conv_layers: 1
num_fc_units: 16
dropout_rate: 0.0001
epoch: 0 total_correct: 7683 loss: 1374.965255498886
Train Accuracy: 0.12805
Test Accuracy: 0.1835
Accuracy: 0.1835


learning rate: 3.2e-05
num_conv_layers: 3
num_fc_units: 475
dropout_rate: 6.554583805713542e-05
epoch: 0 total_correct: 6840 loss: 1380.1562609672546
Train Accuracy: 0.114
Test Accuracy: 0.2045
Accuracy: 0.2045


learning rate: 1.4e-06
num_conv_layers: 2
num_fc_units: 332
dropout_rate: 0.008263500480211641
epoch: 0 total_correct: 5923 loss: 1381.8805747032166
Train Accuracy: 0.09871666666666666
Test Accuracy: 0.098
Accuracy: 0.098


learning rate: 1.4e-04
num_conv_layers: 1
num_fc_units: 369
dropout_rate: 0.00024463712155743455
epoch: 0 total_correct: 48370 loss: 557.3785186186433
Train Accuracy: 0.8061666666666667
Test Accuracy: 0.9056
Accuracy: 0.9056


learning rate: 2.6e-03
num_conv_layers: 2
num_fc_units: 246
dropout_rate: 1.5303097393960503e-05
epoch: 0 total_correct: 50108 loss: 307

In [42]:
model = torch.load(best_model_path)

In [43]:
score(model, train_loader, 100)

-258.5201912654412

In [44]:
score(model, train_loader, 100)

-255.93358079193638

In [45]:
score(model, train_loader, 100)

-1405.1369280804388

In [48]:
score(model, train_loader, 100)

-274.17343593713525

In [49]:
score(model, train_loader, 100)

-269.1172612628513

In [50]:
score(model, train_loader, 100)

-274.4908487728444

In [51]:
score(model, train_loader, 100)

-272.94912093133127

In [53]:
score(model, train_loader, 100)

-267.4807335426352

In [54]:
score(model, train_loader, 100)

-270.02138317752775

In [55]:
score(model, train_loader, 100)

-251.4902082469836

In [58]:
score(model, train_loader, 100)

-274.22823468841784

In [68]:
train_set = torchvision.datasets.FashionMNIST(
    root='./data/FashionMNIST'
    ,train=True
    ,download=True
    ,transform=transforms.Compose([
        transforms.ToTensor()
    ])
)

train_loader = torch.utils.data.DataLoader(train_set, batch_size=100)

In [79]:
from model_utils import Network

In [80]:
network = Network()

In [63]:
# MNISTConvNet Model with MNIST

for i in range(5):
    print(score(model, train_loader, 100))

-252.87908629939508
-253.68628686489285
-262.81342022702245
-250.77947543684232
-266.47151279492357


In [78]:
# MNISTConvNet Model with FashionMNIST

score(model, train_loader, 100)

-174.53300854914806

In [84]:
# DeepLizard Model with FashinMNIST

for i in range(5):
    print(score(network, train_loader, 100))

-130.0591546563784
-129.5577054965591
-130.10801703043032
-130.50272044612768
-130.20478774893365


In [86]:
# DeepLizard Model with MNIST

for i in range(5):
    print(score(network, train_loader, 100))

-140.72732323378915
-140.93314403853495
-140.58060849670062
-140.8401433763754
-141.25414008094714
